In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, HDFStore
import matplotlib.pyplot as plt
# import matplotlib.pyplot as plt
import datetime as dt
import sqlite3
# import pandas_datareader.data as web

import quandl as ql

import stocks.tools as st

In [2]:
cnx        = sqlite3.connect('database/stocks_main.db')
xetra      = pd.read_sql("SELECT * FROM XETRA;", cnx)
currencies = pd.read_sql("SELECT * FROM currencies", cnx)

In [41]:
class keyratios:
    def __init__(self):
        pass
    
    def return_keyratios(self,filename):
        self.load_keyratios(filename)
        return self.keyratios
   
    def _get_title(self):
        with open(self.filename) as f:
            for line in f:
                return line
    
    def load_keyratios(self,filename):
        self.filename = filename
        try:
            self.keyratios = pd.read_csv(filename,skiprows=2)
        except IOError:
            self.keyratios = None
            return
        self._correct_colnames()
        self.keyratios = self._format_keyratios()
    
    def _correct_colnames(self):
        newcols = []
        for i in self.keyratios.columns:
            if i == 'TTM':
                newcols.append(newcols[-1]+1)
                continue
            if 'Unnamed' in i:
                newcols.append('year')
                continue 
            newcols.append(int(i.split('-')[0]))
        self.keyratios.columns = newcols

    def _format_keyratios(self):
        pdf = self.keyratios
        pdf = pdf.transpose()
        
        cols_new     = []
        cols_to_use  = []

        cols_old = np.array(pdf[pdf.index=='year'])[0]
        
        currency = self._find_currency(cols_old)       
        
        for string in cols_old:
    #         print string
            try:
                newstring = string
                newstring = newstring.replace(' ','')
                newstring = newstring.replace('%', '')
                newstring = newstring.replace('Mil','')
                newstring = newstring.replace('*','')
                newstring = newstring.replace('(','')
                newstring = newstring.replace(')','')
                newstring = newstring.replace("'",'')

                for j in currencies.currency:
                    newstring = newstring.replace(j,'')
            except AttributeError:
                newstring = 'NaN'

            if newstring in cols_new:
                newstring = '{0}_{1}'.format(newstring,len(cols_new)+1)
            cols_new.append(newstring)    

        try:
            pdf.columns = cols_new
        except ValueError:
            return cols_old,cols_new
        pdf         = pdf[1:]


        for c in cols_new:
            if '_' not in c and '&' not in c:
                if c not in ['NaN','3-YearAverage','Efficiency','MarginsofSales','Growth',
                             'BalanceSheetItemsin','Liquidity/FinancialHealth','Profitability',
                             'CashFlowRatios']:
                    cols_to_use.append(c)

        pdf = pdf[cols_to_use]
        pdf = pdf.replace(to_replace=",",value="",regex=True)    
        pdf = pdf.astype('float')


        pdf = pdf[[
                    'Shares',
                    'Revenue',
                    'NetIncome',
                    'EBTMargin', 
                    'EarningsPerShare',
                    'Dividends',
                    "TotalStockholdersEquity",
                    'ReturnonEquity',
                    'BookValuePerShare',
                    'OperatingCashFlow',
                    'GrossMargin',
                    'OperatingIncome', 
                    'OperatingMargin',
                    'CapSpending',
                    'FreeCashFlow', 
                    'FreeCashFlowPerShare', 
                    'WorkingCapital', 
                    'COGS',
                    'TaxRate', 
                    'NetMargin',
                    'AssetTurnoverAverage', 
                    'ReturnonAssets',
                    'FinancialLeverageAverage',
                    'ReturnonInvestedCapital', 
                    'InterestCoverage' 
                   ]]

        pdf.currency = currency

        return pdf    
        
    def _find_currency(self,cols_old):
        for string in cols_old:
            if similar('Revenue XXX Mil', string)==0.8:
                cur = string.split(' ')[1]
                self.currency = cur
                return cur

In [42]:
class stocks(keyratios):
    def __init__(self,stocklist,index=0,ISIN=None,verbose=False):
        self.list  = stocklist

        if ISIN:
            self.switch_isin(ISIN)
        else:
            self.switch_index(index)
        self._update_tables()
        
    def switch_isin(self,isin):
        df         = self.list[self.list.ISIN==isin]
        self.index = df.index[0]
        self.name, self.isin, self.ticker = np.array(df)[0]
        self._update_tables()
        
    def switch_index(self,index):
        self.index = index
        df         = self.list[self.list.index==index]
        self.name, self.isin, self.ticker = np.array(df)[0]
#         self._update_tables()
        
    def switch_next(self):
        try:
            self.switch_index(self.index+1)
            self._update_tables()
            self.quote = None
        except IndexError:
            pass

    def _update_tables(self):
#         try:
        self.load_keyratios('ms_keyratios/'+self.ticker+'.csv')
#         except:
#             self.keyratios = None

In [43]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [44]:
def merge_keyratios(isin,ticker):
    nok1 = False
    
    try:
        k1 = keyratios()
        k1.load_keyratios('raw_data/ms_keyratios_06_15/{0}.csv'.format(ticker))
        k1.title = k1._get_title().split('Ratios for')[-1]
        k1.keyratios = k1.keyratios[k1.keyratios.index==2006]
    except FileNotFoundError:
        nok1=True
        
        
    k2 = keyratios()
    k2.load_keyratios('raw_data/ms_keyratios_07_16/{0}_{1}.csv'.format(isin[0:2], ticker))
    k2.title = k2._get_title().split('Ratios for')[-1]
    k2.keyratios = k2.keyratios[k2.keyratios.index!=2017]
    
    if nok1:
        try:
            return k2.keyratios, k2.currency
        except AttributeError:
            return k2.keyratios, "NAN"
        
    if similar(k1.title,k2.title)<0.5:
        print('Names not identical: |{0}|{1}|'.format(k1.title.replace('\n',''),k2.title.replace('\n','')))
        try:
            return k2.keyratios, k2.currency
        except AttributeError:
            return k2.keyratios, "NAN"
        
    return k1.keyratios.append(k2.keyratios), k2.currency

In [33]:
def create_table_for_sql(sobj):
    isin, name, ticker = sobj.isin, sobj.name, sobj.ticker
#     print('isin, name, ticker = {0} {1} {2}'.format(isin,name,ticker))
    
    # merge the keyratio dataframes
    try:
        df, cur = merge_keyratios(isin, ticker)
    except FileNotFoundError:
        return None
        
        
    df.reset_index(inplace=True)
    df=df.rename(columns = {'index':'year'})
    
    # prepare the columns containing isin, name and currency
    isin_array = []
    curr = []
    nama = []
    for i in range(len(df)):
        isin_array.append(isin)
        curr.append(cur)
        nama.append(name)
    
    df = df.assign(name=nama)
    df = df.assign(ISIN=isin_array)
    df = df.assign(currency=curr)

    # change the shape of the columns
    cols = list(df.columns)
    for i in range(0,3):
        cols = cols[-1:] + cols[:-1]

    df = df[cols]
    
    return df

https://www.dataquest.io/blog/python-pandas-databases/

In [34]:
def add_to_db(dbname,sqltable):
    cnx = sqlite3.connect(dbname)

    if sqltable is not None:
        sqltable.to_sql('fundamentals',cnx,if_exists='append',index=False)

    cnx.close()

In [35]:
# s = stocks(xetra)
# s.switch_isin('CH0038863350')
# s.name, s.isin, s.ticker

In [36]:
# sqltable = create_table_for_sql(s)
# sqltable

In [40]:
s = stocks(xetra)

# for i in range(0,len(xetra)):
for i in range(0,100):    
    print('Adding to DB: {0}'.format(s.name))
    try:
        sqltable = create_table_for_sql(s)
        add_to_db('database/test3.db',sqltable)
    except:
        pass
    s.switch_next()

Adding to DB: DP WORLD LTD    DL 2
['Revenue', 'GrossMargin', 'OperatingIncome', 'OperatingMargin', 'NetIncome', 'EarningsPerShare', 'Dividends', 'PayoutRatio', 'Shares', 'BookValuePerShare', 'OperatingCashFlow', 'CapSpending', 'FreeCashFlow', 'FreeCashFlowPerShare', 'WorkingCapital', 'KeyRatios-&gt;Profitability', 'MarginsofSales', 'Revenue_18', 'COGS', 'GrossMargin_20', 'SG&amp;A;', 'R&amp;D;', 'Other', 'OperatingMargin_24', 'NetIntInc&amp;Other', 'EBTMargin', 'Profitability', 'TaxRate', 'NetMargin', 'AssetTurnoverAverage', 'ReturnonAssets', 'FinancialLeverageAverage', 'ReturnonEquity', 'ReturnonInvestedCapital', 'InterestCoverage', 'KeyRatios-&gt;Growth', 'NaN', 'Revenue_38', 'YearoverYear', '3-YearAverage', '5-YearAverage', '10-YearAverage', 'OperatingIncome_43', 'YearoverYear_44', '3-YearAverage_45', '5-YearAverage_46', '10-YearAverage_47', 'NetIncome_48', 'YearoverYear_49', '3-YearAverage_50', '5-YearAverage_51', '10-YearAverage_52', 'EPS', 'YearoverYear_54', '3-YearAverage_55', 

['Revenue', 'GrossMargin', 'OperatingIncome', 'OperatingMargin', 'NetIncome', 'EarningsPerShare', 'Dividends', 'PayoutRatio', 'Shares', 'BookValuePerShare', 'OperatingCashFlow', 'CapSpending', 'FreeCashFlow', 'FreeCashFlowPerShare', 'WorkingCapital', 'KeyRatios-&gt;Profitability', 'MarginsofSales', 'Revenue_18', 'COGS', 'GrossMargin_20', 'SG&amp;A;', 'R&amp;D;', 'Other', 'OperatingMargin_24', 'NetIntInc&amp;Other', 'EBTMargin', 'Profitability', 'TaxRate', 'NetMargin', 'AssetTurnoverAverage', 'ReturnonAssets', 'FinancialLeverageAverage', 'ReturnonEquity', 'ReturnonInvestedCapital', 'InterestCoverage', 'KeyRatios-&gt;Growth', 'NaN', 'Revenue_38', 'YearoverYear', '3-YearAverage', '5-YearAverage', '10-YearAverage', 'OperatingIncome_43', 'YearoverYear_44', '3-YearAverage_45', '5-YearAverage_46', '10-YearAverage_47', 'NetIncome_48', 'YearoverYear_49', '3-YearAverage_50', '5-YearAverage_51', '10-YearAverage_52', 'EPS', 'YearoverYear_54', '3-YearAverage_55', '5-YearAverage_56', '10-YearAverage

['Revenue', 'GrossMargin', 'OperatingIncome', 'OperatingMargin', 'NetIncome', 'EarningsPerShare', 'Dividends', 'PayoutRatio', 'Shares', 'BookValuePerShare', 'OperatingCashFlow', 'CapSpending', 'FreeCashFlow', 'FreeCashFlowPerShare', 'WorkingCapital', 'KeyRatios-&gt;Profitability', 'MarginsofSales', 'Revenue_18', 'COGS', 'GrossMargin_20', 'SG&amp;A;', 'R&amp;D;', 'Other', 'OperatingMargin_24', 'NetIntInc&amp;Other', 'EBTMargin', 'Profitability', 'TaxRate', 'NetMargin', 'AssetTurnoverAverage', 'ReturnonAssets', 'FinancialLeverageAverage', 'ReturnonEquity', 'ReturnonInvestedCapital', 'InterestCoverage', 'KeyRatios-&gt;Growth', 'NaN', 'Revenue_38', 'YearoverYear', '3-YearAverage', '5-YearAverage', '10-YearAverage', 'OperatingIncome_43', 'YearoverYear_44', '3-YearAverage_45', '5-YearAverage_46', '10-YearAverage_47', 'NetIncome_48', 'YearoverYear_49', '3-YearAverage_50', '5-YearAverage_51', '10-YearAverage_52', 'EPS', 'YearoverYear_54', '3-YearAverage_55', '5-YearAverage_56', '10-YearAverage

['Revenue', 'GrossMargin', 'OperatingIncome', 'OperatingMargin', 'NetIncome', 'EarningsPerShare', 'Dividends', 'PayoutRatio', 'Shares', 'BookValuePerShare', 'OperatingCashFlow', 'CapSpending', 'FreeCashFlow', 'FreeCashFlowPerShare', 'WorkingCapital', 'KeyRatios-&gt;Profitability', 'MarginsofSales', 'Revenue_18', 'COGS', 'GrossMargin_20', 'SG&amp;A;', 'R&amp;D;', 'Other', 'OperatingMargin_24', 'NetIntInc&amp;Other', 'EBTMargin', 'Profitability', 'TaxRate', 'NetMargin', 'AssetTurnoverAverage', 'ReturnonAssets', 'FinancialLeverageAverage', 'ReturnonEquity', 'ReturnonInvestedCapital', 'InterestCoverage', 'KeyRatios-&gt;Growth', 'NaN', 'Revenue_38', 'YearoverYear', '3-YearAverage', '5-YearAverage', '10-YearAverage', 'OperatingIncome_43', 'YearoverYear_44', '3-YearAverage_45', '5-YearAverage_46', '10-YearAverage_47', 'NetIncome_48', 'YearoverYear_49', '3-YearAverage_50', '5-YearAverage_51', '10-YearAverage_52', 'EPS', 'YearoverYear_54', '3-YearAverage_55', '5-YearAverage_56', '10-YearAverage

['Revenue', 'GrossMargin', 'OperatingIncome', 'OperatingMargin', 'NetIncome', 'EarningsPerShare', 'Dividends', 'PayoutRatio', 'Shares', 'BookValuePerShare', 'OperatingCashFlow', 'CapSpending', 'FreeCashFlow', 'FreeCashFlowPerShare', 'WorkingCapital', 'KeyRatios-&gt;Profitability', 'MarginsofSales', 'Revenue_18', 'COGS', 'GrossMargin_20', 'SG&amp;A;', 'R&amp;D;', 'Other', 'OperatingMargin_24', 'NetIntInc&amp;Other', 'EBTMargin', 'Profitability', 'TaxRate', 'NetMargin', 'AssetTurnoverAverage', 'ReturnonAssets', 'FinancialLeverageAverage', 'ReturnonEquity', 'ReturnonInvestedCapital', 'InterestCoverage', 'KeyRatios-&gt;Growth', 'NaN', 'Revenue_38', 'YearoverYear', '3-YearAverage', '5-YearAverage', '10-YearAverage', 'OperatingIncome_43', 'YearoverYear_44', '3-YearAverage_45', '5-YearAverage_46', '10-YearAverage_47', 'NetIncome_48', 'YearoverYear_49', '3-YearAverage_50', '5-YearAverage_51', '10-YearAverage_52', 'EPS', 'YearoverYear_54', '3-YearAverage_55', '5-YearAverage_56', '10-YearAverage

...

['Revenue', 'GrossMargin', 'OperatingIncome', 'OperatingMargin', 'NetIncome', 'EarningsPerShare', 'Dividends', 'PayoutRatio', 'Shares', 'BookValuePerShare', 'OperatingCashFlow', 'CapSpending', 'FreeCashFlow', 'FreeCashFlowPerShare', 'WorkingCapital', 'KeyRatios->Profitability', 'MarginsofSales', 'Revenue_18', 'COGS', 'GrossMargin_20', 'SG&A', 'R&D', 'Other', 'OperatingMargin_24', 'NetIntInc&Other', 'EBTMargin', 'Profitability', 'TaxRate', 'NetMargin', 'AssetTurnoverAverage', 'ReturnonAssets', 'FinancialLeverageAverage', 'ReturnonEquity', 'ReturnonInvestedCapital', 'InterestCoverage', 'KeyRatios->Growth', 'NaN', 'Revenue_38', 'YearoverYear', '3-YearAverage', '5-YearAverage', '10-YearAverage', 'OperatingIncome_43', 'YearoverYear_44', '3-YearAverage_45', '5-YearAverage_46', '10-YearAverage_47', 'NetIncome_48', 'YearoverYear_49', '3-YearAverage_50', '5-YearAverage_51', '10-YearAverage_52', 'EPS', 'YearoverYear_54', '3-YearAverage_55', '5-YearAverage_56', '10-YearAverage_57', 'KeyRatios->Ca

['Revenue', 'GrossMargin', 'OperatingIncome', 'OperatingMargin', 'NetIncome', 'EarningsPerShare', 'Dividends', 'PayoutRatio', 'Shares', 'BookValuePerShare', 'OperatingCashFlow', 'CapSpending', 'FreeCashFlow', 'FreeCashFlowPerShare', 'WorkingCapital', 'KeyRatios->Profitability', 'MarginsofSales', 'Revenue_18', 'COGS', 'GrossMargin_20', 'SG&A', 'R&D', 'Other', 'OperatingMargin_24', 'NetIntInc&Other', 'EBTMargin', 'Profitability', 'TaxRate', 'NetMargin', 'AssetTurnoverAverage', 'ReturnonAssets', 'FinancialLeverageAverage', 'ReturnonEquity', 'ReturnonInvestedCapital', 'InterestCoverage', 'KeyRatios->Growth', 'NaN', 'Revenue_38', 'YearoverYear', '3-YearAverage', '5-YearAverage', '10-YearAverage', 'OperatingIncome_43', 'YearoverYear_44', '3-YearAverage_45', '5-YearAverage_46', '10-YearAverage_47', 'NetIncome_48', 'YearoverYear_49', '3-YearAverage_50', '5-YearAverage_51', '10-YearAverage_52', 'EPS', 'YearoverYear_54', '3-YearAverage_55', '5-YearAverage_56', '10-YearAverage_57', 'KeyRatios->Ca

Adding to DB: RANGE RES. LTD
['Revenue', 'GrossMargin', 'OperatingIncome', 'OperatingMargin', 'NetIncome', 'EarningsPerShare', 'Dividends', 'PayoutRatio', 'Shares', 'BookValuePerShare', 'OperatingCashFlow', 'CapSpending', 'FreeCashFlow', 'FreeCashFlowPerShare', 'WorkingCapital', 'KeyRatios-&gt;Profitability', 'MarginsofSales', 'Revenue_18', 'COGS', 'GrossMargin_20', 'SG&amp;A;', 'R&amp;D;', 'Other', 'OperatingMargin_24', 'NetIntInc&amp;Other', 'EBTMargin', 'Profitability', 'TaxRate', 'NetMargin', 'AssetTurnoverAverage', 'ReturnonAssets', 'FinancialLeverageAverage', 'ReturnonEquity', 'ReturnonInvestedCapital', 'InterestCoverage', 'KeyRatios-&gt;Growth', 'NaN', 'Revenue_38', 'YearoverYear', '3-YearAverage', '5-YearAverage', '10-YearAverage', 'OperatingIncome_43', 'YearoverYear_44', '3-YearAverage_45', '5-YearAverage_46', '10-YearAverage_47', 'NetIncome_48', 'YearoverYear_49', '3-YearAverage_50', '5-YearAverage_51', '10-YearAverage_52', 'EPS', 'YearoverYear_54', '3-YearAverage_55', '5-Yea

['Revenue', 'GrossMargin', 'OperatingIncome', 'OperatingMargin', 'NetIncome', 'EarningsPerShare', 'Dividends', 'PayoutRatio', 'Shares', 'BookValuePerShare', 'OperatingCashFlow', 'CapSpending', 'FreeCashFlow', 'FreeCashFlowPerShare', 'WorkingCapital', 'KeyRatios-&gt;Profitability', 'MarginsofSales', 'Revenue_18', 'COGS', 'GrossMargin_20', 'SG&amp;A;', 'R&amp;D;', 'Other', 'OperatingMargin_24', 'NetIntInc&amp;Other', 'EBTMargin', 'Profitability', 'TaxRate', 'NetMargin', 'AssetTurnoverAverage', 'ReturnonAssets', 'FinancialLeverageAverage', 'ReturnonEquity', 'ReturnonInvestedCapital', 'InterestCoverage', 'KeyRatios-&gt;Growth', 'NaN', 'Revenue_38', 'YearoverYear', '3-YearAverage', '5-YearAverage', '10-YearAverage', 'OperatingIncome_43', 'YearoverYear_44', '3-YearAverage_45', '5-YearAverage_46', '10-YearAverage_47', 'NetIncome_48', 'YearoverYear_49', '3-YearAverage_50', '5-YearAverage_51', '10-YearAverage_52', 'EPS', 'YearoverYear_54', '3-YearAverage_55', '5-YearAverage_56', '10-YearAverage

Adding to DB: SUMMIT RES LTD.
['Revenue', 'GrossMargin', 'OperatingIncome', 'OperatingMargin', 'NetIncome', 'EarningsPerShare', 'Dividends', 'PayoutRatio', 'Shares', 'BookValuePerShare', 'OperatingCashFlow', 'CapSpending', 'FreeCashFlow', 'FreeCashFlowPerShare', 'WorkingCapital', 'KeyRatios-&gt;Profitability', 'MarginsofSales', 'Revenue_18', 'COGS', 'GrossMargin_20', 'SG&amp;A;', 'R&amp;D;', 'Other', 'OperatingMargin_24', 'NetIntInc&amp;Other', 'EBTMargin', 'Profitability', 'TaxRate', 'NetMargin', 'AssetTurnoverAverage', 'ReturnonAssets', 'FinancialLeverageAverage', 'ReturnonEquity', 'ReturnonInvestedCapital', 'InterestCoverage', 'KeyRatios-&gt;Growth', 'NaN', 'Revenue_38', 'YearoverYear', '3-YearAverage', '5-YearAverage', '10-YearAverage', 'OperatingIncome_43', 'YearoverYear_44', '3-YearAverage_45', '5-YearAverage_46', '10-YearAverage_47', 'NetIncome_48', 'YearoverYear_49', '3-YearAverage_50', '5-YearAverage_51', '10-YearAverage_52', 'EPS', 'YearoverYear_54', '3-YearAverage_55', '5-Ye